In [ ]:
import pandas as pd
import os
import sys
import boto3
s3_client = boto3.client('s3')
sys.path.append(os.path.expanduser('../'))
from scripts.utils.write_metadata import (
    make_metadata_files, 
    append_metadata
)

In [ ]:
# df = pd.read_csv('Full Data Pipeline Notes - 1_ Pull.csv')
meta_csv = 'https://docs.google.com/spreadsheets/d/1GBvawVFW7C5G7bNjqJFNaPYf8ZQ9tPTZL1WLVgcQkhw/edit?usp=sharing'
# read in first tab of the sheet
df = pd.read_html(meta_csv)[0]
# name columns
df.columns = df.iloc[0]
# drop empty columns
df = df.loc[:, df.columns.notna()]
# drop first two rows
df = df.iloc[2:]
# do some housekeeping
df = df.rename(
    columns={
        'Ideas for AWS naming' : 'Variable',
        'Data' : 'Source Dataset Name',
        'Date of First Pull ' : 'Date Pulled',
        'Data Puller' : 'Data Pulled By',
    }
)
df = df.drop(columns=['Pulled Format','Notes','Link',df.columns[0]])
df = df.fillna('N/A')
pd.set_option('display.max_rows', 10)

df

In [ ]:
make_metadata_files(df)

### Make a fake test function to test out the decorator

In [ ]:
# sample function to show how the decorator works
@append_metadata
def test_metadata_output(x, y, varname=''):
    '''
    Example docstring for sample function.
    '''
    return x * y

In [ ]:
# Call the decorated function
# and append metadata to the generated files
varname = 'climate_fema_nfip_claims' # can pick any valid varname from the spreadsheet
result = test_metadata_output(10, 20, varname=varname)

Download the file so we can check results

In [ ]:
metadata_path = "3_fair_data/metadata/1_pull/"
file_name = f"{varname}_metadata.txt"
obj_name = f"{metadata_path}{file_name}"
s3_client.download_file(
    'ca-climate-index', obj_name, file_name)